In [ ]:
# 1.导入必要库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
# 2.设置超参数和设备
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

In [ ]:
# 3.加载数据
trainloader = torch.utils.data.DataLoader(datasets.MNIST('data', train=True, download=True,
              transform=transforms.Compose([transforms.ToTensor()])), batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(datasets.MNIST('data', train=False, download=True,
              transform=transforms.Compose([transforms.ToTensor()])), batch_size=batch_size, shuffle=True)
print("训练数据形状:", trainloader.dataset.data.shape)
print("测试数据形式:", testloader.dataset.data.shape)

In [ ]:
# 4.创建模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)     # 输入通道1，输出通道6，卷积核5x5
        self.conv2 = nn.Conv2d(6, 16, 5)    # 输入通道6，输出通道16，卷积核5x5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)   # 全连接层，输入16*5*5，输出120
        self.fc2 = nn.Linear(120, 84)   # 全连接层，输入120，输出84
        self.clf = nn.Linear(84, 10)    # 分类层，输入84，输出10

    def forward(self, x):
        # conv1
        x = self.conv1(x)
        x = F.sigmoid(x)    # 激活函数sigmoid()
        x = F.avg_pool2d(x, kernel_size=2, stride=2)    # 平均池化层，kernel=2x2，步长2
        # conv2
        x = self.conv2(x)
        x = F.sigmoid(x)    # 激活函数sigmoid()
        x = F.avg_pool2d(x, kernel_size=2, stride=2)    # 平均池化层，2x2，步长2
        # 展开，从第1维开始展开
        x = x.view(x.size(0), -1)
        # 全连接层1
        x = self.fc1(x)
        x = F.sigmoid(x)    # 激活函数sigmoid()
        # 全连接层2
        x = self.fc2(x)
        x = F.sigmoid(x)    # 激活函数sigmoid()
        # 分类层
        x = self.clf(x)
        return x


model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

epochs = 30
accs, losses = [], []

In [ ]:
# 5.训练和测试
for epoch in range(epochs):
    model.train()
    for batch_idx, (x, y) in enumerate(trainloader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = F.cross_entropy(out, y)
        loss.backward()
        optimizer.step()

    # ---- 测试 ----
    model.eval()
    correct = 0
    testloss = 0
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(testloader):
            x, y = x.to(device), y.to(device)
            out = model(x)
            testloss += F.cross_entropy(out, y).item()
            pred = out.max(dim=1)[1]
            correct += pred.eq(y).sum().item()

    acc = correct / len(testloader.dataset)
    testloss /= (batch_idx + 1)
    accs.append(acc)
    losses.append(testloss)
    print('epoch:{}, loss:{:.4f}, acc:{:.4f}'.format(epoch, testloss, acc))

In [ ]:
# 画 LOSS 和 ACC 曲线
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(losses, label='Test Loss')
plt.title("Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()

plt.subplot(1,2,2)
plt.plot(accs, label='Test Acc')
plt.title("Accuracy Curve")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid()

plt.savefig("res1/loss_acc_curve.png")
plt.show()

In [ ]:
# 计算并绘制 Confusion Matrix
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = out.max(dim=1)[1]
        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().numpy())

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig("res1/confusion_matrix.png")
plt.show()

In [ ]:
# 可视化每层卷积 + 池化的 Feature Map
model.eval()
with torch.no_grad():
    x, y = next(iter(testloader))
    x = x.to(device)
    # conv1 feature map
    f1 = F.sigmoid(model.conv1(x))
    p1 = F.avg_pool2d(f1, 2)
    # conv2 feature map
    f2 = F.sigmoid(model.conv2(p1))
    p2 = F.avg_pool2d(f2, 2)

def show_feature_maps(feature_maps, title, filename):
    fm = feature_maps[0].cpu().numpy()   # 取 batch 的第一个样本
    C = fm.shape[0]

    cols = 6
    rows = int(np.ceil(C / cols))

    plt.figure(figsize=(12, rows * 2))
    plt.suptitle(title)

    for i in range(C):
        plt.subplot(rows, cols, i+1)
        plt.imshow(fm[i], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"res1/{filename}.png")
    plt.show()

show_feature_maps(f1, "Conv1 Feature Maps", "conv1_feature")
show_feature_maps(p1, "Pool1 Feature Maps", "pool1_feature")
show_feature_maps(f2, "Conv2 Feature Maps", "conv2_feature")
show_feature_maps(p2, "Pool2 Feature Maps", "pool2_feature")